In [54]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.misc as sm
data_dir = '/home/lioruzan/pixel-cnn/data/letters_data'
import pickle as pkl

with open(os.path.join(data_dir,'letters_test.pkl'),'rb') as f:
    test_letters= pkl.load(f)

In [20]:
len(test_letters['masks'])

147

In [55]:
from scipy.ndimage import measurements as me
rotations = np.zeros(len(test_letters['masks']))
c=0
for m in test_letters['masks']:
    m = m[:,:,0]
    xm,ym = me.center_of_mass(m)
    m[int(xm),int(ym)] = 0
    xm,ym=int(xm)//16,int(ym)//16
    if xm==0 and ym==0:
        rotations[c] = 0
    if xm==0 and ym==1:
        rotations[c] = 1
    if xm==1 and ym==1:
        rotations[c] = 2
    if xm==1 and ym==0:
        rotations[c] = 3
#     plt.imshow(m*255)
#     plt.show()
#     plt.imshow(np.rot90(m,k=rotations[c]))
#     plt.show()
#     print(xm,ym)
    c += 1


In [56]:
test_letters['labels'] = rotations
with open(os.path.join(data_dir,'letters_test.pkl'),'wb') as f:
    pkl.dump(test_letters, f)

In [66]:
print((rotations==0).sum()%32)
print((rotations==1).sum()%20)
print((rotations==2).sum()%4)
print((rotations==3).sum()%12)


4
1
2
0


In [3]:
for m in test_letters['masks']:
    sm.imshow(m*255)
    break

In [28]:
''' MAKE IMAGES '''
import scipy.misc as sm
import numpy as np

imgs = [sm.imread(file,mode='RGB') for file in file_list]

# resized_imgs = [sm.imresize(im,(32,32)) for im in imgs]

# resized_imgs = [im[np.newaxis,:,:,:] for im in resized_imgs]
# images = np.vstack(resized_imgs)
# labels = np.array(label_list)

[ 192.  167.    3.] [ 47.07156859  29.70098797   0.        ]


In [29]:
''' IMAGE STATS '''
shapes=[o.shape for o in imgs]
print(np.median(shapes, axis=0), np.mean(shapes, axis=0), np.std(shapes, axis=0))
print(np.max(shapes,axis=0),np.min(shapes,axis=0))
print(np.argmax(shapes, axis=0), np.argmin(shapes,axis=0))

[ 192.  167.    3.] [ 208.19724556  168.49743231    3.        ] [ 47.07156859  29.70098797   0.        ]
